# Import der notwendigen Module


- ``tensorflow`` für das Neuronale Netzwerk
- ``tensorflow_datasets`` für den Datensatz

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Laden des Datensatzes

In [2]:
dataset = tfds.load('cats_vs_dogs', split='train', shuffle_files=True)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]WARNING:absl:1738 images were corrupted and were skipped


Dataset cats_vs_dogs downloaded and prepared to C:\Users\nic-e\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [6]:
type(dataset)

tensorflow.python.data.ops.dataset_ops._OptionsDataset